In [44]:
#IMPORT Libaries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import pyproj


In [18]:
#IMPORT Data 

#AMTOVZ_CSV_LV95 (BFS-Nr)
df_gemeinden = pd.read_csv("./AMTOVZ_CSV_LV95.csv", sep =';', header=0)
#Einwohner_proGemeinde_Tabelle_raw (CODE_REGION)
df_einwohner = pd.read_csv("./einwohner_Gemeinden_V2.csv", sep =';', header=0)

gdf_gemeinden_cent = gpd.read_file('Gemeinden_centroidv2.shp')

In [19]:
df_gemeinden.head()

,Ortschaftsname,PLZ,Zusatzziffer,Gemeindename,BFS-Nr,Kantonskürzel,E,N,Sprache,Validity
0,Aeugst am Albis,8914,0,Aeugst am Albis,1,ZH,2679402.872,1235842.010,de,2008-07-01
1,Aeugstertal,8914,2,Aeugst am Albis,1,ZH,2679815.372,1237404.310,de,2008-07-01
2,Zwillikon,8909,0,Affoltern am Albis,2,ZH,2675280.133,1238108.286,de,2008-07-01
3,Affoltern am Albis,8910,0,Affoltern am Albis,2,ZH,2676852.012,1236929.718,de,2008-07-01
4,Bonstetten,8906,0,Bonstetten,3,ZH,2677412.150,1241078.278,de,2008-07-01


In [20]:
df_einwohner.head(5)

,PERIOD_REF,PERIOD_COMP,CODE_REGION,REGION,INDICATORS,UNIT_MES,VALUE,STATUS
0,2019,NaN,CH,Schweiz,Ind_01_01,UM_6,8606033,A
1,2019,NaN,1,Aeugst am Albis,Ind_01_01,UM_6,1981,A
2,2019,NaN,2,Affoltern am Albis,Ind_01_01,UM_6,12303,A
3,2019,NaN,3,Bonstetten,Ind_01_01,UM_6,5572,A
4,2019,NaN,4,Hausen am Albis,Ind_01_01,UM_6,3751,A


In [21]:
gdf_gemeinden_cent.drop(['FK_LOCALIT', 'ZIP_ID', 'ADDITIONAL', 'STATUS', 'INMODIFICA','MODIFIED', 'SHAPE_LEN'  ], axis=1, inplace=True)
gdf_gemeinden_cent['E'] = gdf_gemeinden_cent.geometry.x
gdf_gemeinden_cent['N'] = gdf_gemeinden_cent.geometry.y
gdf_gemeinden_cent.head()

,ZIP4,SHAPE_AREA,geometry,E,N
0,6315,10098585.901907,POINT (2692305.490 1222243.448),2.692305e+06,1.222243e+06
1,7425,3655017.94528,POINT (2751731.433 1173495.858),2.751731e+06,1.173496e+06
2,2087,3707807.6549885,POINT (2568211.753 1209392.779),2.568212e+06,1.209393e+06
3,3963,18469726.944762,POINT (2604236.729 1131303.915),2.604237e+06,1.131304e+06
4,2575,1431062.234648,POINT (2581793.052 1213928.816),2.581793e+06,1.213929e+06


In [22]:
print("MAX: ", gdf_gemeinden_cent['SHAPE_AREA'].max(), " MIN: ", gdf_gemeinden_cent['SHAPE_AREA'].min())


"""
# Plot histogram of 'SHAPE_AREA' values
plt.hist(gdf_gemeinden_cent['SHAPE_AREA'], bins=30, edgecolor='black')
plt.title('Distribution of Shape Area')
plt.xlabel('Shape Area')
plt.ylabel('Frequency')
plt.show()
"""

MAX:  999520.240417  MIN:  10000184.5693775


"\n# Plot histogram of 'SHAPE_AREA' values\nplt.hist(gdf_gemeinden_cent['SHAPE_AREA'], bins=30, edgecolor='black')\nplt.title('Distribution of Shape Area')\nplt.xlabel('Shape Area')\nplt.ylabel('Frequency')\nplt.show()\n"

In [23]:
df_einwohner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173 entries, 0 to 2172
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PERIOD_REF   2173 non-null   int64  
 1   PERIOD_COMP  0 non-null      float64
 2   CODE_REGION  2173 non-null   object 
 3   REGION       2173 non-null   object 
 4   INDICATORS   2173 non-null   object 
 5   UNIT_MES     2173 non-null   object 
 6   VALUE        2173 non-null   int64  
 7   STATUS       2173 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 135.9+ KB


In [24]:
df_gemeinden.rename(columns={'PLZ': 'ZIP4'}, inplace=True)

# Convert 'ZIP4' column to object type in df_gemeinden
df_gemeinden['ZIP4'] = df_gemeinden['ZIP4'].astype(str)

merged_df_gem = pd.merge(gdf_gemeinden_cent, df_gemeinden [['ZIP4','BFS-Nr', 'Gemeindename']], on='ZIP4', how='left')

merged_df_gem = merged_df_gem.drop_duplicates(subset='ZIP4', keep='first')

In [25]:
merged_df_gem.head()

,ZIP4,SHAPE_AREA,geometry,E,N,BFS-Nr,Gemeindename
0,6315,10098585.901907,POINT (2692305.490 1222243.448),2.692305e+06,1.222243e+06,1371,Sattel
6,7425,3655017.94528,POINT (2751731.433 1173495.858),2.751731e+06,1.173496e+06,3663,Masein
7,2087,3707807.6549885,POINT (2568211.753 1209392.779),2.568212e+06,1.209393e+06,6451,Cornaux
9,3963,18469726.944762,POINT (2604236.729 1131303.915),2.604237e+06,1.131304e+06,6239,Icogne
16,2575,1431062.234648,POINT (2581793.052 1213928.816),2.581793e+06,1.213929e+06,497,Lüscherz


In [26]:
#Join datasets by adding VALUE from Einwohner_proGemeinde_Tabelle_raw by CODE_REGION and BFS-Nr

# Rename 'CODE_REGION' column to 'BFS-Nr' in merged_df_gem
df_einwohner.rename(columns={'CODE_REGION': 'BFS-Nr'}, inplace=True)

# Convert 'BFS-Nr' column to object type in df_gemeinden
merged_df_gem['BFS-Nr'] = merged_df_gem['BFS-Nr'].astype(str)

# Merge the two DataFrames on the common column 'BFS-Nr'
merged_df = pd.merge(merged_df_gem, df_einwohner[['BFS-Nr', 'VALUE']], on='BFS-Nr', how='left')
merged_df.rename(columns={'VALUE': 'EINWOHNER'}, inplace=True)


In [27]:
merged_df.head(40)

,ZIP4,SHAPE_AREA,geometry,E,N,BFS-Nr,Gemeindename,EINWOHNER
0,6315,10098585.901907,POINT (2692305.490 1222243.448),2.692305e+06,1.222243e+06,1371,Sattel,1925.0
1,7425,3655017.94528,POINT (2751731.433 1173495.858),2.751731e+06,1.173496e+06,3663,Masein,498.0
2,2087,3707807.6549885,POINT (2568211.753 1209392.779),2.568212e+06,1.209393e+06,6451,Cornaux,1558.0
3,3963,18469726.944762,POINT (2604236.729 1131303.915),2.604237e+06,1.131304e+06,6239,Icogne,593.0
4,2575,1431062.234648,POINT (2581793.052 1213928.816),2.581793e+06,1.213929e+06,497,Lüscherz,551.0
5,7175,35518114.312382,POINT (2712806.885 1180052.378),2.712807e+06,1.180052e+06,3985,Sumvitg,1121.0
6,1610,3112746.373563,POINT (2552847.891 1157988.746),2.552848e+06,1.157989e+06,5805,Oron,5661.0
7,9314,3226872.847651,POINT (2743448.446 1266523.953),2.743448e+06,1.266524e+06,4411,Egnach,4699.0
8,9220,11951706.9995747,POINT (2736973.643 1262286.475),2.736974e+06,1.262286e+06,3202,Muolen,1220.0
9,4566,1846692.421909,POINT (2613136.760 1224149.303),2.613137e+06,1.224149e+06,2520,Halten,836.0


In [36]:
merged_df = gpd.GeoDataFrame(merged_df, geometry="geometry")

In [37]:
#Select Towns in a certain radius

"""
#RADIUS ATTEMPT -> NOT USED
# LV95 projection definition
lv95 = pyproj.Proj('epsg:2056')

# Convert LV95 coordinates of the center to Cartesian coordinates
center_x_buochs, center_y_buochs = 2673116,1201977 # 
center_x_cartesian, center_y_cartesian = lv95(center_x_buochs, center_y_buochs, inverse=True)

# Assuming you have the radius of the circle in LV95 coordinates
radius_lv95 = 5000  # Example LV95 radius

# Convert the radius from meters in LV95 to degrees in the Cartesian coordinate system
radius_cartesian = radius_lv95 / (2 * 6371000) * (180 / 3.14159)

# Create a Point representing the center of the circle in Cartesian coordinates
center_point = Point(center_x_cartesian, center_y_cartesian)

# Create a buffer around the center point with the given radius to define the circular region
region_polygon = center_point.buffer(radius_cartesian)
"""
#------Entlebuch-----------
min_x_entle, min_y_entle = 2618336, 1205234  # minimum coordinates
max_x_entle, max_y_entle = 2632820, 1215590  # maximum coordinates


region_polygon_entle = Polygon([(min_x_entle, min_y_entle), 
                         (max_x_entle, min_y_entle), 
                         (max_x_entle, max_y_entle), 
                         (min_x_entle, max_y_entle)]) 

# Select rows within the specified region
#town_entle = merged_df[merged_df.geometry.within(region_polygon_entle)]
#town_entle = town_entle.drop(columns='geometry')

#town_entle = town_entle.drop_duplicates(subset='Gemeindename', keep='first')
#town_entle.to_csv('towns_entles.csv', index=False)

#------Zweisimmen-----------
min_x_zwei, min_y_zwei = 2585050, 1147922  # minimum coordinates
max_x_zwei, max_y_zwei = 2599534, 1158278  # maximum coordinates

region_polygon_zwei = Polygon([(min_x_zwei, min_y_zwei), 
                         (max_x_zwei, min_y_zwei), 
                         (max_x_zwei, max_y_zwei), 
                         (min_x_zwei, max_y_zwei)]) 

# Select rows within the specified region
#town_zwei = merged_df[merged_df.geometry.within(region_polygon_zwei)]
#town_zwei = town_zwei.drop(columns='geometry')

#town_zwei = town_zwei.drop_duplicates(subset='Gemeindename', keep='first')
#town_zwei.to_csv('towns_zwei.csv', index=False)

#------Beromunester------
min_x_bero, min_y_bero = 2651054, 1220378  # minimum coordinates
max_x_bero, max_y_bero = 2665538, 1230734  # maximum coordinates


region_polygon_bero = Polygon([(min_x_bero, min_y_bero), 
                         (max_x_bero, min_y_bero), 
                         (max_x_bero, max_y_bero), 
                         (min_x_bero, max_y_bero)]) 

# Select rows within the specified region
#town_bero = merged_df[merged_df.geometry.within(region_polygon_bero)]
#town_bero = town_bero.drop(columns='geometry')

#town_bero = town_bero.drop_duplicates(subset='Gemeindename', keep='first')
#town_bero.to_csv('towns_bero.csv', index=False)

#------WIL SG------
min_x_wil, min_y_wil = 2713761, 1253732  # minimum coordinates
max_x_wil, max_y_wil = 2728245, 1264088  # maximum coordinates


region_polygon_wil = Polygon([(min_x_wil, min_y_wil), 
                         (max_x_wil, min_y_wil), 
                         (max_x_wil, max_y_wil), 
                         (min_x_wil, max_y_wil)]) 

#town_wil = merged_df[merged_df.geometry.within(region_polygon_wil)]
#town_wil= town_wil.drop(columns='geometry')

#town_wil = town_wil.drop_duplicates(subset='Gemeindename', keep='first')
#town_wil.to_csv('towns_wil.csv', index=False)

#------Grenchen------
min_x_gren, min_y_gren = 2590444, 1219944  # minimum coordinates
max_x_gren, max_y_gren = 2604928, 1230300  # maximum coordinates


region_polygon_gren = Polygon([(min_x_gren, min_y_gren), 
                         (max_x_gren, min_y_gren), 
                         (max_x_gren, max_y_gren), 
                         (min_x_gren, max_y_gren)]) 

town_gren = merged_df[merged_df.geometry.within(region_polygon_gren)]
town_gren = town_gren.drop(columns='geometry')

town_gren = town_gren.drop_duplicates(subset='Gemeindename', keep='first')
town_gren.to_csv('towns_gren.csv', index=False)
#------BUOCHS-----------
min_x_buochs, min_y_buochs = 2664022, 1197812  # minimum coordinates
max_x_buochs, max_y_buochs = 2685712, 1213400  # maximum coordinates


region_polygon_buochs = Polygon([(min_x_buochs, min_y_buochs), 
                         (max_x_buochs, min_y_buochs), 
                         (max_x_buochs, max_y_buochs), 
                         (min_x_buochs, max_y_buochs)]) 
  

# Select rows within the specified region
#town_buochs = merged_df[merged_df.geometry.within(region_polygon_buochs)]
#town_buochs = town_buochs.drop(columns='geometry')

#town_buochs = town_buochs.drop_duplicates(subset='Gemeindename', keep='first')
#town_buochs.to_csv('towns_buochs.csv', index=False)

#-----------------------

#------SCENENAME--------


In [30]:
town_zwei.head()

,ZIP4,SHAPE_AREA,E,N,BFS-Nr,Gemeindename,EINWOHNER
1326,3777,12766363.05782,2.589744e+06,1.151608e+06,794,Zweisimmen,3040.0
1580,1657,9052920.138655,2.587590e+06,1.157221e+06,791,Boltigen,1252.0
1701,3778,17269208.0233335,2.587170e+06,1.152095e+06,843,Saanen,6817.0
3124,3772,31027145.5699395,2.596394e+06,1.149457e+06,793,St. Stephan,1330.0


In [107]:
merged_df = merged_df.drop(columns='geometry')

In [112]:
#EXPORT 

# Export to CSV
merged_df.to_csv('towns_all.csv', index=False)

## SwissTLM Regio Data set Names

In [38]:
#Datasource: https://ogd.swisstopo.admin.ch/ch.swisstopo.swisstlmregio

In [89]:
gdf_gemeinden = gpd.read_file('swissTLMRegio_NamedLocation.shp')

In [90]:
gdf_gemeinden.head()

,OBJECTID,OBJORIG,DATEOFCHNG,NLN1,NLN2,ALTITUDE,NAMN1,NAMN2,UUID,OBJORIG_YE,OBJVAL,geometry
0,1,NDB,2009-04-03,GER,N_A,627,Ankenreute,N_A,{4DB10515-1AC1-477D-AEF0-4C515C10CD88},2009,KOrtschaft,POINT (2773058.000 1306212.000)
1,2,ERM,2017-09-05,FRE,N_A,290,Guebwiller,N_A,{FD474E9D-2A44-4F22-B19E-4BA2C23B6B9A},2017,HOrtschaft1,POINT (2583373.000 1306061.000)
2,3,Divers,2014-12-03,FRE,N_A,620,Col du Mont de Fourche,N_A,{33035E2C-FE7D-4FED-B787-F678BAE4EB6A},2014,Strassenpass,POINT (2540660.000 1306139.000)
3,4,NDB,2009-04-03,GER,N_A,578,Schreckensee,N_A,{363C7EB9-B7E0-4AA1-A262-CBD51B271D08},2009,MOrtschaft,POINT (2757698.000 1306165.000)
4,5,NDB,2009-04-03,GER,N_A,207,Weinstetten,N_A,{3A0E366D-1D89-49D4-8410-F3BAB97F5055},2009,KOrtschaft,POINT (2611981.000 1306113.000)


In [91]:
# List of columns to drop
columns_to_drop = ['OBJECTID', 'OBJORIG', 'DATEOFCHNG', 'NLN1', 'NLN2', 'NAMN2','OBJORIG_YE']

# Drop the columns
gdf = gdf_gemeinden.drop(columns=columns_to_drop)

# Filter the GeoDataFrame to only include rows where 'OBJVAL' is in a specific category
merged_df = gdf[gdf['OBJVAL'].isin(['GOrtschaft', 'HOrtschaft1', 'MOrtschaft1','HOrtschaft2', 'HOrtschaft3'])]#,'MOrtschaft'
merged_df['E'] = merged_df.geometry.x
merged_df['N'] = merged_df.geometry.y
merged_df.head()

C:\Users\thier\anaconda3\envs\Advanced_GIS_23\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\thier\anaconda3\envs\Advanced_GIS_23\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,ALTITUDE,NAMN1,UUID,OBJVAL,geometry,E,N
1,290,Guebwiller,{FD474E9D-2A44-4F22-B19E-4BA2C23B6B9A},HOrtschaft1,POINT (2583373.000 1306061.000),2583373.0,1306061.0
17,891,Hinterzarten,{F6F24B07-4441-458E-A543-18B16D393EE1},GOrtschaft,POINT (2649880.000 1305844.000),2649880.0,1305844.0
22,249,Issenheim,{7F1D1916-B5F3-486F-872D-1DF6B66803B4},GOrtschaft,POINT (2585888.000 1305761.000),2585888.0,1305761.0
41,219,Lehen,{D1FE331E-1DB2-4373-BFEF-CAB294BC7C22},GOrtschaft,POINT (2627029.000 1318639.000),2627029.0,1318639.0
42,271,Herdern,{269A2C2D-18B9-4E93-A993-3869FB31EBF8},GOrtschaft,POINT (2631053.000 1317737.000),2631053.0,1317737.0


In [92]:
#Select Towns in a certain radius

"""
#RADIUS ATTEMPT -> NOT USED
# LV95 projection definition
lv95 = pyproj.Proj('epsg:2056')

# Convert LV95 coordinates of the center to Cartesian coordinates
center_x_buochs, center_y_buochs = 2673116,1201977 # 
center_x_cartesian, center_y_cartesian = lv95(center_x_buochs, center_y_buochs, inverse=True)

# Assuming you have the radius of the circle in LV95 coordinates
radius_lv95 = 5000  # Example LV95 radius

# Convert the radius from meters in LV95 to degrees in the Cartesian coordinate system
radius_cartesian = radius_lv95 / (2 * 6371000) * (180 / 3.14159)

# Create a Point representing the center of the circle in Cartesian coordinates
center_point = Point(center_x_cartesian, center_y_cartesian)

# Create a buffer around the center point with the given radius to define the circular region
region_polygon = center_point.buffer(radius_cartesian)
"""
#------Entlebuch-----------
min_x_entle, min_y_entle = 2618336, 1205234  # minimum coordinates
max_x_entle, max_y_entle = 2632820, 1215590  # maximum coordinates


region_polygon_entle = Polygon([(min_x_entle, min_y_entle), 
                         (max_x_entle, min_y_entle), 
                         (max_x_entle, max_y_entle), 
                         (min_x_entle, max_y_entle)]) 

# Select rows within the specified region
#town_entle = merged_df[merged_df.geometry.within(region_polygon_entle)]
#town_entle = town_entle.drop(columns='geometry')

#town_entle = town_entle.drop_duplicates(subset='Gemeindename', keep='first')
#town_entle.to_csv('towns_entle_cent.csv', index=False)

#------Zweisimmen-----------
min_x_zwei, min_y_zwei = 2585050, 1147922  # minimum coordinates
max_x_zwei, max_y_zwei = 2599534, 1158278  # maximum coordinates

region_polygon_zwei = Polygon([(min_x_zwei, min_y_zwei), 
                         (max_x_zwei, min_y_zwei), 
                         (max_x_zwei, max_y_zwei), 
                         (min_x_zwei, max_y_zwei)]) 

# Select rows within the specified region
#town_zwei = merged_df[merged_df.geometry.within(region_polygon_zwei)]
#town_zwei = town_zwei.drop(columns='geometry')

#town_zwei = town_zwei.drop_duplicates(subset='Gemeindename', keep='first')
#town_zwei.to_csv('towns_zwei_cent.csv', index=False)

#------Beromunester------
min_x_bero, min_y_bero = 2651054, 1220378  # minimum coordinates
max_x_bero, max_y_bero = 2665538, 1230734  # maximum coordinates


region_polygon_bero = Polygon([(min_x_bero, min_y_bero), 
                         (max_x_bero, min_y_bero), 
                         (max_x_bero, max_y_bero), 
                         (min_x_bero, max_y_bero)]) 

# Select rows within the specified region
#town_bero = merged_df[merged_df.geometry.within(region_polygon_bero)]
#town_bero = town_bero.drop(columns='geometry')

#town_bero = town_bero.drop_duplicates(subset='Gemeindename', keep='first')
#town_bero.to_csv('towns_bero_cent.csv', index=False)

#------WIL SG------
min_x_wil, min_y_wil = 2713761, 1253732  # minimum coordinates
max_x_wil, max_y_wil = 2728245, 1264088  # maximum coordinates


region_polygon_wil = Polygon([(min_x_wil, min_y_wil), 
                         (max_x_wil, min_y_wil), 
                         (max_x_wil, max_y_wil), 
                         (min_x_wil, max_y_wil)]) 

#town_wil = merged_df[merged_df.geometry.within(region_polygon_wil)]
#town_wil= town_wil.drop(columns='geometry')

#town_wil = town_wil.drop_duplicates(subset='Gemeindename', keep='first')
#town_wil.to_csv('towns_wil_cent.csv', index=False)

#------Grenchen------
min_x_gren, min_y_gren = 2590444, 1219944  # minimum coordinates
max_x_gren, max_y_gren = 2604928, 1230300  # maximum coordinates


region_polygon_gren = Polygon([(min_x_gren, min_y_gren), 
                         (max_x_gren, min_y_gren), 
                         (max_x_gren, max_y_gren), 
                         (min_x_gren, max_y_gren)]) 

#town_gren = merged_df[merged_df.geometry.within(region_polygon_gren)]
#town_gren = town_gren.drop(columns='geometry')

#town_gren = town_gren.drop_duplicates(subset='Gemeindename', keep='first')
#town_gren.to_csv('towns_gren_cent.csv', index=False)

#------BUOCHS-----------
min_x_buochs, min_y_buochs = 2664022, 1197812  # minimum coordinates
max_x_buochs, max_y_buochs = 2685712, 1213400  # maximum coordinates


region_polygon_buochs = Polygon([(min_x_buochs, min_y_buochs), 
                         (max_x_buochs, min_y_buochs), 
                         (max_x_buochs, max_y_buochs), 
                         (min_x_buochs, max_y_buochs)]) 
  

# Select rows within the specified region
town_buochs = merged_df[merged_df.geometry.within(region_polygon_buochs)]
town_buochs = town_buochs.drop(columns='geometry')

#town_buochs = town_buochs.drop_duplicates(subset='Gemeindename', keep='first')
town_buochs.to_csv('towns_buochs_cent.csv', index=False)

#-----------------------

In [ ]:
#merged_df.to_csv('towns_all.csv', index=False)